# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
weather_df = pd.read_csv('output/CityWeatherOutputs.csv')
weather_df.drop('Unnamed: 0', axis=1)
# weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,avarua,-21.2078,-159.7750,73.45,88,30,3.44,CK,1661752944
1,ushuaia,-54.8000,-68.3000,42.46,70,0,2.30,AR,1661752683
2,leningradskiy,69.3833,178.4167,39.81,82,65,5.93,RU,1661752945
3,atuona,-9.8000,-139.0333,77.22,74,33,21.50,PF,1661752945
4,sao filipe,14.8961,-24.4956,76.60,83,95,14.50,CV,1661752945
...,...,...,...,...,...,...,...,...,...
569,soligalich,59.0778,42.2864,74.35,47,99,4.63,RU,1661753187
570,alofi,-19.0595,-169.9187,73.29,83,40,3.44,NU,1661753187
571,nanakuli,21.3906,-158.1547,79.11,61,0,5.75,US,1661753188
572,yerbogachen,61.2767,108.0108,47.64,68,95,13.53,RU,1661753188


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
#configure
gmaps.configure(api_key = g_key)

#Use the Lat and Lng as locations and Humidity as the weight.
lat_lng_locations = weather_df[['Lat','Lng']].astype(float)
humidity = weather_df['Humidity'].astype(float)

#mapping add heatmap layer
layout = {'width': '1000px', 'height': '600px'}
fig = gmaps.figure(map_type = 'HYBRID', layout= layout)

heat_layer = gmaps.heatmap_layer(lat_lng_locations, weights= humidity, max_intensity= max(weather_df['Humidity']), point_radius= 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='600px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

A max temperature lower than 80 degrees but higher than 70.


Wind speed less than 10 mph.


Zero cloudiness.

In [57]:
desired_weather_df = weather_df.loc[(weather_df['Max Temp'] <= 80) & (weather_df['Max Temp'] >= 70) &
                                    (weather_df['Wind Speed'] <= 10) & (weather_df['Cloudiness'] < 1)]
desired_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,sheboygan,43.7333,-87.9334,72.37,94,0,5.75,US,1661752902
32,32,ascension,30.2169,-90.9001,78.08,96,0,0.00,US,1661752957
56,56,kars,40.6020,43.0950,73.27,27,0,3.91,TR,1661752930
63,63,shahr-e kord,32.3256,50.8644,71.69,10,0,4.34,IR,1661752970
116,116,newala,-10.9431,39.2787,70.90,51,0,3.13,TZ,1661752994
117,117,kostyantynivka,48.5277,37.7069,77.92,58,0,8.03,UA,1661752994
136,136,mayo,38.8876,-76.5119,77.99,92,0,4.00,US,1661753003
167,167,rafsanjan,30.4067,55.9939,75.40,9,0,5.66,IR,1661753016
270,270,volodarka,49.5248,29.9122,75.88,32,0,6.20,UA,1661753060
282,282,portland,45.5234,-122.6762,71.58,67,0,8.05,US,1661752206


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [58]:
hotel_df = desired_weather_df
hotel_df.reset_index(drop= True, inplace= True)
hotel_df.drop('Unnamed: 0', axis=1)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sheboygan,43.7333,-87.9334,72.37,94,0,5.75,US,1661752902
1,ascension,30.2169,-90.9001,78.08,96,0,0.00,US,1661752957
2,kars,40.6020,43.0950,73.27,27,0,3.91,TR,1661752930
3,shahr-e kord,32.3256,50.8644,71.69,10,0,4.34,IR,1661752970
4,newala,-10.9431,39.2787,70.90,51,0,3.13,TZ,1661752994
5,kostyantynivka,48.5277,37.7069,77.92,58,0,8.03,UA,1661752994
6,mayo,38.8876,-76.5119,77.99,92,0,4.00,US,1661753003
7,rafsanjan,30.4067,55.9939,75.40,9,0,5.66,IR,1661753016
8,volodarka,49.5248,29.9122,75.88,32,0,6.20,UA,1661753060
9,portland,45.5234,-122.6762,71.58,67,0,8.05,US,1661752206


In [62]:
#add hotel name column
hotel_df['Hotel Name'] = ' '
                #hotel_df
#link for api info search note to self
# https://developers.google.com/maps/documentation/javascript/places#find_place_from_query
# https://googlemaps.github.io/google-maps-services-python/docs/
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

#create parameters
params = {"radius": 5000, "types": "lodging", 
          "keyword": "hotel", "key": g_key}

#loop through data frame to grab nearby hotel info using iterrows
for i, j in hotel_df.iterrows():
    #get locations
    lats = j['Lat']
    lngs = j['Lng']
    
    #add the location into the parameters created above
    params['location'] = f'{lats},{lngs}'
    
    #build url
    response = requests.get(url, params= params).json()
    #print(json.dumps(response, indent=4, sort_keys=True))
    
    result = response['results']
    # print(json.dumps(result[0], indent=4, sort_keys=True))
    # print(json.dumps(result[0]['name'], indent=4, sort_keys=True))
    # break
    
    #try to get hotel if in the area | except if it doesnt have any nearby
    try:
        print('getting hotel')
        hotel_df.loc[i, 'Hotel Name'] = result[0]['name']
    except:
        print('hotel not found')
        
        
hotel_df

C:\Users\David\AppData\Local\Temp\ipykernel_9136\1418647084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ' '


getting hotel
getting hotel
getting hotel
getting hotel
getting hotel
getting hotel
getting hotel
hotel not found
getting hotel
getting hotel
hotel not found
getting hotel
getting hotel
getting hotel
hotel not found
getting hotel
getting hotel
getting hotel
getting hotel
getting hotel
getting hotel
getting hotel
getting hotel
getting hotel
getting hotel
getting hotel
getting hotel
getting hotel
hotel not found


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,30,sheboygan,43.7333,-87.9334,72.37,94,0,5.75,US,1661752902,Baymont by Wyndham Plymouth
1,32,ascension,30.2169,-90.9001,78.08,96,0,0.00,US,1661752957,"Holiday Inn Express & Suites Gonzales, an IHG ..."
2,56,kars,40.6020,43.0950,73.27,27,0,3.91,TR,1661752930,Cheltikov Hotel
3,63,shahr-e kord,32.3256,50.8644,71.69,10,0,4.34,IR,1661752970,Parsian Hotel
4,116,newala,-10.9431,39.2787,70.90,51,0,3.13,TZ,1661752994,Country Lodge
5,117,kostyantynivka,48.5277,37.7069,77.92,58,0,8.03,UA,1661752994,Hotelʹ Komfort
6,136,mayo,38.8876,-76.5119,77.99,92,0,4.00,US,1661753003,
7,167,rafsanjan,30.4067,55.9939,75.40,9,0,5.66,IR,1661753016,Hilan Hotel
8,270,volodarka,49.5248,29.9122,75.88,32,0,6.20,UA,1661753060,
9,282,portland,45.5234,-122.6762,71.58,67,0,8.05,US,1661752206,River's Edge Hotel


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map


# Display figure
